In [1]:
pwd

'/p/project/raise-ctp2/cern/particleflow/nbs'

In [2]:
import sys
sys.path.append("/p/project/raise-ctp2/cern/particleflow/mlpf")

In [3]:
# mlpf pipeline imports
import os
import yaml
import json
import datetime
import glob
import random
import platform
import numpy as np
from pathlib import Path
import click
from tqdm import tqdm
import shutil
from functools import partial
import shlex
import subprocess

import tensorflow as tf
from tensorflow.keras import mixed_precision
import tensorflow_addons as tfa
import keras_tuner as kt

# from tfmodel.model_setup import (
#     make_model,
#     configure_model_weights,
#     LearningRateLoggingCallback,
#     prepare_callbacks,
#     FlattenedCategoricalAccuracy,
#     eval_model,
#     freeze_model,
# )

# from tfmodel.utils import (
#     get_lr_schedule,
#     get_optimizer,
#     create_experiment_dir,
#     get_strategy,
#     make_weight_function,
#     load_config,
#     compute_weights_invsqrt,
#     compute_weights_none,
#     get_train_val_datasets,
#     targets_multi_output,
#     get_dataset_def,
#     prepare_val_data,
#     set_config_loss,
#     get_loss_dict,
#     parse_config,
#     get_best_checkpoint,
#     delete_all_but_best_checkpoint,
#     get_tuner,
# )

from tfmodel.lr_finder import LRFinder
from tfmodel.callbacks import CustomTensorBoard
from tfmodel import hypertuning

import ray
from ray import tune
from ray.tune.integration.keras import TuneReportCheckpointCallback
from ray.tune.integration.tensorflow import DistributedTrainableCreator
from ray.tune.logger import TBXLoggerCallback
from ray.tune import Analysis, ExperimentAnalysis

2023-04-06 14:53:14.130858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 14:53:27.710704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /p/software/jurecadc/stages/2022/software/OpenMPI/4.1.2-GCC-11.2.0/lib:/p/software/jurecadc/stages/2022/software/libevent/2.1.12-GCCcore-11.2.0/lib:/p/software/jurecadc/stages/2022/software/hwloc/2.5.0-GCCcore-11.2.0/lib:/p/software/jurecadc/stages/2022/software/libpciaccess/0.16-GCCcore-11.2.0/lib:/p/software/jurecadc/stages/2022/software/libxml2/2.9.10-GCCcore-11.2.0/lib:/p/software/jurecadc/stag

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:
from matplotlib import rc_file
rc_file("my_matplotlib_rcparams")

In [6]:
ray_results_folder = "/p/project/raise-ctp2/cern/ray_results/"  # Main folder containing all ray experiments

In [7]:
ll $ray_results_folder

total 2240
drwxr-sr-x 1840 wulff1 524288 Feb 20 09:50 asha_scikit_samples400/
drwxr-sr-x   11 wulff1  16384 Feb 16 16:18 clic_gnn_scan/
drwxr-sr-x    8 wulff1  16384 Feb 16 20:31 clic_transformer_bs_scan/
drwxr-sr-x   11 wulff1  16384 Feb 15 16:38 clic_transformer_scan/
drwxr-sr-x  458 wulff1 131072 Apr  4 12:35 clic_transformer_search_asha_hyperopt_n500/
drwxr-sr-x  502 wulff1 131072 Mar  1 13:36 clic_transformer_search_asha_n500/
drwxr-sr-x  234 wulff1  65536 Jun  1  2022 cmsgen_asha_epochs100/
drwxr-sr-x  202 wulff1  65536 Jun  7  2022 cmsgen_asha_epochs50/
drwxr-sr-x  471 wulff1 131072 Aug  3  2022 delphes_svr_dataset/
drwxr-sr-x   22 wulff1  16384 Sep 21  2022 event_loss_scan/
drwxr-sr-x   26 wulff1  16384 Sep 22  2022 event_loss_scan6/
drwxrwxrwx  112 wulff1  16384 May 29  2022 tune_result_dir/


In [16]:
# Which experiments to analyze
exp_dir_list = [# ray_results_folder + "cmsgen_asha_epochs50",
                # ray_results_folder + "cmsgen_asha_epochs100",
                # ray_results_folder + "cmsgen_asha_scikit_epochs50_samples400_OneGPUPerTrial",
                # ray_results_folder + "cmsgen_asha_scikit_epochs50_samples200_FourGPUPerTrial",
                # ray_results_folder + "clic_transformer_search_asha_n500",
                ray_results_folder + "clic_transformer_search_asha_hyperopt_n500",
               ]

In [17]:
default_mode = "min"
default_metric = "val_loss"
expanalysis_list = []
for exp_dir in exp_dir_list:
    expanalysis_list.append(ExperimentAnalysis(exp_dir, default_metric=default_metric, default_mode=default_mode))

In [18]:
def robust_get_df(analysis):
    try:
        result_df = analysis.dataframe()
    except IndexError:
        result_df = analysis.results_df.dropna(axis=0, how="all")
    return result_df

In [19]:
print("Time from start to timestamp:")
for expanalysis in expanalysis_list:
    res_df = robust_get_df(expanalysis)
    exp_dir = Path(res_df['logdir'][0]).parent
    stats = expanalysis.stats() 
    print("{:.2f}".format((stats["timestamp"] - stats["start_time"]) / 60 / 60), "h", exp_dir)

Time from start to timestamp:


2.98 h /p/project/raise-ctp2/cern/ray_results/clic_transformer_search_asha_hyperopt_n500


In [20]:
res_df.keys()

Index(['charge_loss', 'cls_loss', 'cos_phi_loss', 'energy_loss', 'eta_loss',
       'learning_rate', 'loss', 'pt_loss', 'sin_phi_loss', 'val_charge_loss',
       'val_cls_loss', 'val_cos_phi_loss', 'val_energy_loss', 'val_eta_loss',
       'val_loss', 'val_pt_loss', 'val_sin_phi_loss', 'val_jet_wd',
       'val_jet_iqr', 'val_jet_med', 'val_met_wd', 'val_met_iqr',
       'val_met_med', 'time_this_iter_s', 'should_checkpoint', 'done',
       'timesteps_total', 'episodes_total', 'training_iteration', 'trial_id',
       'experiment_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname',
       'node_ip', 'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'warmup_time', 'config/batch_multiplier',
       'config/hidden_dim', 'config/lr', 'config/num_heads',
       'config/num_layers_decoder_cls', 'config/num_layers_decoder_reg',
       'config/num_layers_encoder', 'config/num_random_features',
       'config/out_hidden_dim', 'config/out_num_layers', 'lo

In [21]:
def summarize_best_results(expanalysis_list):
    summ = []
    for expanalysis in expanalysis_list:
        res_df = robust_get_df(expanalysis)
        exp_dir = Path(res_df['logdir'][0]).parent

        dd = expanalysis.best_result
        val_reg_loss = sum([dd["val_{}_loss".format(l)] for l in ["energy", "pt", "eta", "sin_phi", "cos_phi", "charge"]])
        reg_loss = sum([dd["{}_loss".format(l)] for l in ["energy", "pt", "eta", "sin_phi", "cos_phi", "charge"]])

        conf = expanalysis.best_result.pop("config")

        expanalysis.best_result.update({"val_reg_loss": val_reg_loss, "reg_loss": reg_loss})
        expanalysis.best_result.update({x: conf[x] for x in conf})

        summ.append(pd.DataFrame(expanalysis.best_result, index=[exp_dir]))
        expanalysis.best_result.update({"config": conf})
    return pd.concat(summ)
    

In [22]:
summary = summarize_best_results(expanalysis_list)

In [23]:
interesting = ["loss", "val_loss",
               "cls_loss", "val_cls_loss",
               "reg_loss", "val_reg_loss",
               # 'cls_acc_unweighted', 'val_cls_acc_weighted',
               # 'cls_acc_unweighted', 'val_cls_acc_unweighted',
              ]
interesting += [x for x in expanalysis.best_result["config"].keys()]
summary.sort_values(default_metric)
summary[interesting].transpose()

,/p/project/raise-ctp2/cern/ray_results/clic_transformer_search_asha_hyperopt_n500
loss,7.921819
val_loss,7.834934
cls_loss,0.050746
val_cls_loss,0.049924
reg_loss,0.390448
val_reg_loss,0.389944
lr,0.000100
batch_multiplier,10.000000
num_layers_encoder,1.000000
num_layers_decoder_reg,4.000000
